## 수치 미분을 이용한 심층 신경망 학습

In [7]:
import time
import numpy as np

## 유틸리티 함수

In [8]:
epsilon = 0.0001

def _t(x):
    return np.transpose(x)

def _m(A, B):
    return np.matmul(A, B)

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def mean_squared_error(h, y):
    return 1 / 2 * np.mean(np.square(h - y))

## 뉴런 구현

In [9]:
%config IPCompleter.greedy=True

In [15]:
class Neuron:
    def __init__(self, W, b, a):
        # Model Parameters
        self.W = W
        self.b = b
        self.a = a
        
        # Gradients
        self.dW = np.zeros_like(self.W)
        self.db = np.zeros_like(self.b)

    def __call__(self, x):
        return self.a(_m(_t(self.W), x) + self.b)

## 심층신경망 구현

In [17]:
class DNN:
    def __init__(self, hidden_depth, num_neuron, num_input, num_output, activation=sigmoid):
        def init_var(i, o):
            return np.random.normal(0.0, 0.01, (i, o)), np.zeros((o,))

        self.sequence = list()
        # First hidden layer
        W, b = init_var(num_input, num_neuron)
        self.sequence.append(Neuron(W, b, activation))
        
        # Hidden layers
        for _ in range(hidden_depth):
            W, b = init_var(num_neuron, num_neuron)
            self.sequence.append(Neuron(W, b, activation))

        # Output layer
        W, b = init_var(num_neuron, num_output)
        self.sequence.append(Neuron(W, b, activation))

    def __call__(self, x):
        for layer in self.sequence:
            x = layer(x)
        return x

    def calc_gradient(self, x, y, loss_func):
        def get_new_sequence(layer_index, new_neuron):
            new_sequence = list()
            for i, layer in enumerate(self.sequence):
                if i == layer_index:
                    new_sequence.append(new_neuron)
                else:
                    new_sequence.append(layer)
            return new_sequence
        
        def eval_sequence(x, sequence):
            for layer in sequence:
                x = layer(x)
            return x
        
        loss = loss_func(self(x), y)
        
        for layer_id, layer in enumerate(self.sequence): # iterate layer
            for w_i, w in enumerate(layer.W): # iterate W (row)
                for w_j, ww in enumerate(w): # iterate w (col)
                    W = np.copy(layer.W)
                    W[w_i][w_j] = ww + epsilon
                    
                    new_neuron = Neuron(W, layer.b, layer.a)
                    new_seq = get_new_sequence(layer_id, new_neuron)
                    h = eval_sequence(x, new_seq)
                    
                    num_grad = (loss_func(h, y) - loss) / epsilon
                    layer.dW[w_i][w_j] = num_grad
                    
                for b_i, bb in enumerate(layer.b):
                    b = np.copy(layer.b)
                    b[b_i] = bb + epsilon
                    
                    new_neuron = Neuron(layer.W, b, layer.a)
                    new_seq = get_new_sequence(layer_id, new_neuron)
                    h = eval_sequence(x, new_seq)
                    
                    num_grad = (loss_func(h, y) - loss) / epsilon
                    layer.db[b_i] = num_grad
        
        return loss

## 경사하강 학습법

In [18]:
def gradient_descent(network, x, y, loss_obj, alpha=0.01):
    loss = network.calc_gradient(x, y, loss_obj)
    for layer in network.sequence:
        layer.W += -alpha * layer.dW
        layer.b += -alpha * layer.db
    return loss

## 동작 테스트

In [19]:
x = np.random.normal(0.0, 1.0, (10,))
y = np.random.normal(0.0, 1.0, (2,))

dnn = DNN(hidden_depth=5, num_neuron=32, num_input=10, num_output=2, activation=sigmoid)

t = time.time()
for epoch in range(100):
    loss = gradient_descent(dnn, x, y, mean_squared_error, 0.01)
    print('Epoch {}: Test loss {}'.format(epoch, loss))
print('{} seconds elapsed.'.format(time.time() - t))

Epoch 0: Test loss 1.1124140560515952
Epoch 1: Test loss 1.1061694039762475
Epoch 2: Test loss 1.0999680119497117
Epoch 3: Test loss 1.0938128429727172
Epoch 4: Test loss 1.0877067292299283
Epoch 5: Test loss 1.0816523666640014
Epoch 6: Test loss 1.0756523104089377
Epoch 7: Test loss 1.069708971084974
Epoch 8: Test loss 1.0638246119423536
Epoch 9: Test loss 1.0580013468408613
Epoch 10: Test loss 1.0522411390375814
Epoch 11: Test loss 1.0465458007544344
Epoch 12: Test loss 1.0409169934859597
Epoch 13: Test loss 1.0353562290068061
Epoch 14: Test loss 1.029864871034192
Epoch 15: Test loss 1.024444137492984
Epoch 16: Test loss 1.019095103335337
Epoch 17: Test loss 1.0138187038579076
Epoch 18: Test loss 1.0086157384669556
Epoch 19: Test loss 1.00348687483432
Epoch 20: Test loss 0.9984326533911289
Epoch 21: Test loss 0.9934534921092135
Epoch 22: Test loss 0.9885496915172962
Epoch 23: Test loss 0.9837214399048085
Epoch 24: Test loss 0.9789688186667571
Epoch 25: Test loss 0.9742918077470061
Ep